In [3]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")


Setup was successful.


In [24]:
instruments = []
instruments.append("C2_WIND_LTD")
instruments.append("C2_SOLAR_CO")
instruments.append("C2_GREEN_ENERGY_ETF")
'''instruments.append("C1_GAS_INC")
instruments.append("C1_OIL_CORP")
instruments.append("C1_FOSSIL_FUEL_ETF")'''

'instruments.append("C1_GAS_INC")\ninstruments.append("C1_OIL_CORP")\ninstruments.append("C1_FOSSIL_FUEL_ETF")'

In [20]:
# connect to exchange
e = Exchange()
a = e.connect()


2021-09-25 22:16:19,916 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [23]:
# Get order book
def get_highest_basket_bid(basket):
    for bid in basket.bids:
        if bid.volume >= 2:
            return bid
    
def get_lowest_basket_ask(basket):
    for ask in basket.asks:
        if ask.volume >= 2:
            return ask

def get_cash():
    positions = e.get_positions_and_cash()
    cash = 0
    for p in positions:
        cash += positions[p]["cash"]
    return cash

import time
def makeMoney():
    
    start_time = time.time()
    books = []
    positions = e.get_positions()
    books = [e.get_last_price_book(x) for x in instruments]
    if len(books) < 3:
        print("Nothing 1.")
        return get_cash()
    if len(books[2].bids) < 2 or len(books[2].asks) < 2:
        print("Nothing 2.")
        return get_cash()
    basket_highest_bid = get_highest_basket_bid(books[2])
    basket_lowest_ask = get_lowest_basket_ask(books[2])
    if len(books[0].asks) < 2 or len(books[1].asks) < 2:
        print("Nothing 3.")
        return get_cash()
    if len(books[0].bids) < 2 or len(books[1].bids) < 2:
        print("Nothing 4.")
        return get_cash()
    
    basket_sell_profit = basket_highest_bid.price*2 - (books[0].asks[1].price + books[1].asks[1].price) # first_clean_lowest_ask, second_clean_lowest_ask 
    basket_buy_profit = (books[0].bids[1].price + books[1].bids[1].price) - basket_lowest_ask.price*2
    
    if basket_sell_profit > 0:
        clean1_buy_result = e.insert_order(instruments[0], price=books[0].asks[1].price, volume=1, side='bid', order_type='ioc')
        clean2_buy_result = e.insert_order(instruments[1], price=books[1].asks[1].price, volume=1, side='bid', order_type='ioc')
        basket_sell_result = e.insert_order(instruments[2], price=basket_highest_bid.price, volume=2, side='ask', order_type='ioc')
        
    
    elif basket_buy_profit > 0:
        basket_buy_result = e.insert_order(instruments[2], price=basket_lowest_ask.price, volume=2, side='bid', order_type='ioc')
        clean1_sell_result = e.insert_order(instruments[0], price=books[0].bids[1].price, volume=1, side='ask', order_type='ioc')
        clean2_sell_result = e.insert_order(instruments[1], price=books[1].bids[1].price, volume=1, side='ask', order_type='ioc')
    print(f"Basket sell profit: {basket_sell_profit}")
    print(f"Basket buy profit: {basket_buy_profit}")
    #print("--- %s seconds ---" % (time.time() - start_time))
    return get_cash()

In [70]:
from time import sleep
currentMoney = makeMoney()
newMoney = makeMoney()
change = 1
while change < 5:
    newMoney = makeMoney()
    if newMoney != currentMoney:
        print(f"Start: {currentMoney}, end: {newMoney}, change: {newMoney-currentMoney}")
        change+=1
        currentMoney = newMoney
    sleep(0.05)
print(f"Start: {currentMoney}, end: {newMoney}, change: {newMoney-currentMoney}")

Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 3.
Nothing 3.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 2.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 3.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.
Nothing 4.

KeyboardInterrupt: 

In [12]:
books

[None, None, None]

In [192]:
books = [e.get_last_price_book(x) for x in instruments]
positions = e.get_positions_and_cash()
basket_highest_bid = get_highest_basket_bid(books[2])
basket_loewst_ask = get_lowest_basket_ask(books[2])
e.insert_order(instruments[2], price=basket_loewst_ask.price, volume=1, side='ask', order_type='ioc')
e.insert_order(instruments[2], price=basket_highest_bid.price, volume=1, side='bid', order_type='ioc')

{'volume': 1, 'cash': -908.9999999999898}
{'volume': 0, 'cash': -793.9999999999898}


In [125]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [89]:
# Returns all current positions with cash invested
e = Exchange()
a = e.connect()

positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

2021-09-25 22:32:27,959 [asyncio   ] [MainThread  ] Using selector: EpollSelector


C1_GAS_INC {'volume': 0, 'cash': -0.3999999999999915}
C1_OIL_CORP {'volume': 0, 'cash': -0.09999999999999432}
C1_FOSSIL_FUEL_ETF {'volume': 0, 'cash': -0.799999999999983}
C2_GREEN_ENERGY_ETF {'volume': 1, 'cash': -136.90000000000023}
C2_SOLAR_CO {'volume': -45, 'cash': 5620.603999999999}
C2_WIND_LTD {'volume': -45, 'cash': 3744.365999999998}


In [90]:
get_cash()

9226.769999999997

In [38]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
# connect to exchange
e = Exchange()
a = e.connect()

print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

2021-09-25 22:21:01,586 [asyncio   ] [MainThread  ] Using selector: EpollSelector


{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 15, 'C2_SOLAR_CO': -15, 'C2_WIND_LTD': -15}
{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': 0, 'C2_WIND_LTD': 0}


2021-09-25 22:21:01,761 [client    ] [Thread-16   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [159]:
from math import ceil
from enum import Enum


class Instrument(Enum):
    Stock1 = 0
    Stock2 = 1
    Basket = 2


exchange = None
books = None



def get_asks_for_volume(instrument, volume=500):
    global books
    total_prices = [0 for x in range(volume+1)]
    current_volume = 0
    for ask in books[instrument].asks:
        total_prices[:ask.volume+1] = (ask.price, current_volume)
        current_volume = ask.volume+1
    return total_prices


def get_bids_for_volume(instrument, volume=500):
    global books
    total_prices = [0 for x in range(volume+1)]
    current_volume = 0
    books = books[instrument]
    for bid in books.bids:
        total_prices[:bid.volume+1] = (bid.price, current_volume)
        current_volume = bid.volume+1
    return total_prices


def get_best_ask_for_volume(instrument, volume):
    total_price = 0
    total_prices = get_asks_for_volume(instrument, volume)
    end = len(total_prices)
    start = total_prices[end][1]
    while start != 0:
        total_price += total_prices[end][0]*(end-start)
        end = start
        start = total_prices[end][1]
    total_price += total_prices[end][0]*(end)
    return total_price


def get_best_bids_for_volume(instrument, volume):
    total_price = 0
    total_prices = get_bids_for_volume(instrument, volume)
    end = len(total_prices)
    start = total_prices[end][1]
    while start != 0:
        total_price += total_prices[end][0]*(end-start)
        end = start
        start = total_prices[end][1]
    total_price += total_prices[end][0]*(end)
    return total_price


def get_best_current_ask(books, instrument):
    global instruments
    return books[instrument].asks[1]


def get_best_current_bid(books, instrument):
    return books[instrument].bids[1]


def ioc_or_limit_sell(price_when_bought, books, instrument):  # Ako smo od nekog kupili
    return "LIMIT" if get_best_current_bid(books, instrument) < price_when_bought else "IOC"


def ioc_or_limit_buy(price_when_sold, books, instrument):    # Ako smo nekom dužni
    return "LIMIT" if get_best_current_ask(books, instrument) < price_when_sold else "IOC"


def make_ioc_order(instrument, price, vrsta, volume):
    global exchange
    global instruments
    exchange.insert_order(instruments[instrument], price=price,
                          volume=volume, side=vrsta, order_type="ioc")


def make_ioc_order_at_current_best(instrument, vrsta, volume):
    global exchange, books, instruments
    best = get_best_current_ask(
        books, instrument) if type == 'bid' else get_best_current_bid(books, instrument)
    exchange.insert_order(instruments[instrument], price=best.price,
                          volume=abs(volume), side=vrsta, order_type="ioc")


def make_limit_order(instrument, price, vrsta, volume):
    global exchange, instruments
    return exchange.insert_order(instruments[instrument], price=price, volume=volume, side=vrsta, order_type="limit")


def delete_limit_order(instrument, order_id):
    global exchange
    return exchange.delete_order(instrument_id=instrument, order_id=order_id)


def delete_limit_orders(instrument):
    global exchange
    return exchange.delete_orders(instrument_id=instrument)


def check_position(instrument):
    global exchange, instruments
    return exchange.get_positions()[instruments[instrument]]


def calculate_buy_basket_profit(basket_ask, stock1_bid, stock2_bid, minimumVolume=0):
    if minimumVolume > 0:
        return (minimumVolume*stock1_bid.price + minimumVolume*stock2_bid.price)-(basket_ask.price*minimumVolume*2)
    return (stock1_bid.price*stock1_bid.volume + stock2_bid.price*stock2_bid.volume)-(basket_ask.price*basket_ask.volume*2)


def calculate_sell_basket_profit(basket_ask, stock1_bid, stock2_bid, minimumVolume=0):
    if minimumVolume > 0:
        return (basket_ask.price*minimumVolume*2)-(minimumVolume*stock1_bid.price + minimumVolume*stock2_bid.price)
    return (basket_ask.price*basket_ask.volume*2)-(stock1_bid.price*stock1_bid.volume + stock2_bid.price*stock2_bid.volume)


def calculate_best_buy_basket_profit(best_basket_ask, best_stock1_bid, best_stock2_bid):
    basket_volume = max(best_basket_ask.volume, 2)

    stock1_volume = best_stock1_bid.volume
    stock2_volume = best_stock2_bid.volume

    if stock1_volume == stock2_volume and basket_volume == (stock1_volume + stock2_volume):
        return calculate_buy_basket_profit(best_basket_ask, best_stock1_bid, best_stock2_bid)

    elif basket_volume > (stock1_volume + stock2_volume):
        minimum_stock_volume = min(stock1_volume, stock2_volume)
        minimum_volume_profit = calculate_buy_basket_profit(
            best_basket_ask, best_stock1_bid, best_stock2_bid, minimum_stock_volume)
        maximise_volume_profit = calculate_buy_basket_profit(best_basket_ask, get_best_bids_for_volume(Instrument.Stock1.value, volume=ceil(
            basket_volume/2)), get_best_bids_for_volume(Instrument.Stock2.value, volume=ceil(basket_volume/2)))
        if minimum_volume_profit >= maximise_volume_profit:
            return minimum_volume_profit, "minimum_volume"
        else:
            return maximise_volume_profit, "maximise_volume"


def calculate_best_sell_basket_profit(best_basket_bid, best_stock1_ask, best_stock2_ask):
    basket_volume = max(best_basket_bid.volume, 2)

    stock1_volume = best_stock1_ask.volume
    stock2_volume = best_stock2_ask.volume

    if stock1_volume == stock2_volume and basket_volume == (stock1_volume + stock2_volume):
        return calculate_sell_basket_profit(best_basket_bid, best_stock1_ask, best_stock2_ask)

    elif basket_volume > (stock1_volume + stock2_volume):
        minimum_stock_volume = min(stock1_volume, stock2_volume)
        minimum_volume_profit = calculate_sell_basket_profit(
            best_basket_bid, best_stock1_ask, best_stock2_ask, minimum_stock_volume)
        maximise_volume_profit = calculate_sell_basket_profit(best_basket_bid, get_best_ask_for_volume(Instrument.Stock1.value, volume=ceil(
            basket_volume/2)), get_best_ask_for_volume(Instrument.Stock2.value, volume=ceil(basket_volume/2)))
        if minimum_volume_profit >= maximise_volume_profit:
            return minimum_volume_profit, "minimum_volume"
        else:
            return maximise_volume_profit, "maximise_volume"


def calculate_profit(books):
    best_basket_ask = get_best_current_ask(books, Instrument.Basket.value)
    best_stock1_ask = get_best_current_ask(books, Instrument.Stock1.value)
    best_stock2_ask = get_best_current_ask(books, Instrument.Stock2.value)

    best_basket_bid = get_best_current_bid(books, Instrument.Basket.value)
    best_stock1_bid = get_best_current_bid(books, Instrument.Stock1.value)
    best_stock2_bid = get_best_current_bid(books, Instrument.Stock2.value)

    buy_basket_profit = calculate_buy_basket_profit(best_basket_ask, best_stock1_bid, best_stock2_bid)
    sell_basket_profit = calculate_sell_basket_profit(best_basket_bid, best_stock1_ask, best_stock2_ask)
    how_to_order = "minimum_volume"
    best_basket_bid.volume = min(25, best_basket_bid.volume)
    best_basket_ask.volume = min(25, best_basket_ask.volume)
    if buy_basket_profit > sell_basket_profit:
        if how_to_order == "minimum_volume":
            minimum_stock_volume = min(
                best_stock1_bid.volume, best_stock2_bid.volume, 50)
            stock1_sell = make_ioc_order(
                Instrument.Stock1.value, best_stock1_bid.price, "ask", minimum_stock_volume)
            stock2_sell = make_ioc_order(
                Instrument.Stock2.value, best_stock2_bid.price, "ask", minimum_stock_volume)
            basket_buy = make_ioc_order(
                Instrument.Basket.value, best_basket_ask.price, "bid", minimum_stock_volume)
        else:
            basket_buy = make_ioc_order(
                Instrument.Basket.value, best_basket_ask.price, "bid", best_basket_ask.volume)
            stock1_position = check_position(Instrument.Stock1.value)
            stock2_position = check_position(Instrument.Stock2.value)
            while abs(stock1_position) != best_stock1_bid.volume:
                make_ioc_order_at_current_best(
                    Instrument.Stock1.value, 'ask', stock1_position-best_stock1_bid.volume)
            while abs(stock2_position) != best_stock2_bid.volume:
                make_ioc_order_at_current_best(
                    Instrument.Stock2.value, 'ask', stock2_position-best_stock2_bid.volume)
    else:
        if how_to_order == "minimum_volume":
            minimum_stock_volume = min(
                best_stock1_ask.volume, best_stock2_ask.volume)
            basket_sell = make_ioc_order(
                Instrument.Basket.value, best_basket_bid.price, "ask", minimum_stock_volume)
            stock1_buy = make_ioc_order(
                Instrument.Stock1.value, best_stock1_ask.price, "bid", minimum_stock_volume)
            stock2_buy = make_ioc_order(
                Instrument.Stock2.value, best_stock2_ask.price, "bid", minimum_stock_volume)

        else:
            basket_sell = make_ioc_order(
                Instrument.Basket.value, best_basket_bid.price, "ask", best_basket_bid.volume)
            stock1_position = check_position(Instrument.Stock1.value)
            stock2_position = check_position(Instrument.Stock2.value)
            while abs(stock1_position) != best_stock1_ask.volume:
                make_ioc_order_at_current_best(
                    Instrument.Stock1.value, 'bid', stock1_position-best_stock1_ask.volume)
            while abs(stock2_position) != best_stock2_ask.volume:
                make_ioc_order_at_current_best(
                    Instrument.Stock2.value, 'bid', stock2_position-best_stock2_ask.volume)


In [160]:
exchange = Exchange()
a = exchange.connect()
instruments=[]
instruments.append("C2_WIND_LTD")
instruments.append("C2_SOLAR_CO")
instruments.append("C2_GREEN_ENERGY_ETF")

2021-09-25 22:49:17,740 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-09-25 22:49:17,949 [client    ] [Thread-30   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [161]:
def liquidate_assets():
    best_basket_ask = get_best_current_ask(books, Instrument.Basket.value)
    best_stock1_ask = get_best_current_ask(books, Instrument.Stock1.value)
    best_stock2_ask = get_best_current_ask(books, Instrument.Stock2.value)

    best_basket_bid = get_best_current_bid(books, Instrument.Basket.value)
    best_stock1_bid = get_best_current_bid(books, Instrument.Stock1.value)
    best_stock2_bid = get_best_current_bid(books, Instrument.Stock2.value)
    
    stock1_position = check_position(Instrument.Stock1.value)
    stock2_position = check_position(Instrument.Stock2.value)
    basket_position = check_position(Instrument.Basket.value)
    
    if stock1_position > 0:
        make_ioc_order_at_current_best(
                    Instrument.Stock1.value, 'ask', abs(stock1_position))
    elif stock1_position < 0:
        make_ioc_order_at_current_best(
                    Instrument.Stock1.value, 'bid', abs(stock1_position))
    if stock2_position > 0:
        make_ioc_order_at_current_best(
                    Instrument.Stock2.value, 'ask', abs(stock2_position))
    elif stock2_position < 0:
        make_ioc_order_at_current_best(
                    Instrument.Stock2.value, 'bid', abs(stock2_position))
    if basket_position > 0:
        make_ioc_order_at_current_best(
                    Instrument.Basket.value, 'ask', abs(basket_position))
    elif basket_position < 0:
        make_ioc_order_at_current_best(
                    Instrument.Basket.value, 'bid', abs(basket_position))

In [168]:
books = [exchange.get_last_price_book(x) for x in instruments]
calculate_profit(books)
print(get_cash())
exchange.get_positions()

4664.635999999997


{'C1_GAS_INC': 0,
 'C1_OIL_CORP': 0,
 'C1_FOSSIL_FUEL_ETF': 0,
 'C2_GREEN_ENERGY_ETF': 50,
 'C2_SOLAR_CO': -50,
 'C2_WIND_LTD': -50}

In [ ]:
from time import sleep
liquid = False
while not liquid:
    books = [exchange.get_last_price_book(x) for x in instruments]
    try:
        liquidate_assets()
    except:
        continue
    #print(get_cash())
    for position, value in exchange.get_positions().items():
        liquid = True
        if value != 0:
            liquid = False
    sleep(0.05)
print(get_cash())

In [166]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(exchange.get_positions())
for s, p in exchange.get_positions().items():
    if p > 0:
        exchange.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        exchange.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(exchange.get_positions())

{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': -50, 'C2_WIND_LTD': -50}
{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': 0, 'C2_WIND_LTD': 0}


In [167]:
print(get_cash())

-655.3640000000028


In [ ]:
def get_cash():
    positions = exchange.get_positions_and_cash()
    cash = 0
    for p in positions:
        cash += positions[p]["cash"]
    return cash